# dynprops - Python Dynamic Properties
Support for complex table and TSV structures

## Revision History
* 0.1.0 - Initial commit
* 0.1.1 - Fix [issue](https://github.com/hsolbrig/dynprops/issues/1) with non `'_'` properties
* 0.1.2 - Add `reify` option 
* 0.1.3 - Switch to using the csv module for `heading()` and `row()` functions
* 0.1.4 - Use `reify` in attribute retrieve
* 0.1.5 - Two reify spots (DRY failure)
* 0.2.0 - Fixed Issue #2, added reify bypass
* 0.2.1 - Fix unexpected issue with csv library

## Introduction
The `dynprops` package supports three basic features:
* Creation of sets properties that can be emitted in columnar form (tsv, csv, ...) or as dictionaries targeted at SQL tools
* Global 'singleton' properties for columns that are invariant over time
* "Dynamic properties" - properties whose values can either be objects, functions or methods

## Requirements
* The `dynobject` package requires Python 3.6 or later as it uses the 3.6 annotation features (`var: type [= value]`) 

## Installation


In [25]:
! pip install dynprops --upgrade -q

## Usage
### Declaring properties:

Properties are declared as types, using either `Global`, meaning that the property is a singleton or
`Local`, meaning that the property can be class or instance level.

In [26]:
from datetime import datetime
from dynprops import DynProps, Global, Local, Parent, heading, row, as_dict


# The base -- Three global properties - a string, an integer and a date with a dynamic(!) value
class BaseElements(DynProps):
    key_part1: Global[str]
    key_part2: Global[int]
    key_part3: Global[datetime] = datetime.now      # Function


Dynamic properties can be used to generate csv/tsv/?sv headings

In [27]:
DynProps._separator = ','      # Default is tab
print(heading(BaseElements))


key_part1,key_part2,key_part3


An extension to the base with one more global property and two locals


In [28]:
class ShortTable(BaseElements):
    key_part4: Global[str]
    val_part1: Local[str] = "@"
    val_part2: Local[int] = 0

    def __init__(self, vp2: int) -> None:
        self.val_part2 = vp2
print(heading(ShortTable))  

key_part1,key_part2,key_part3,key_part4,val_part1,val_part2


We can now set the values of the global components. Globals must be set at the level of the declaring class
(i.e. `ShortTable.key_part2 = 1` or `BaseElements().key_part_2 = 1` will not work. 

The `row` function prints the value of an instance

In [29]:
BaseElements.key_part1 = "production run"
BaseElements.key_part2 = 17

ShortTable.val_part1 = "Unit"
print(row(ShortTable(3)))
print(row(ShortTable(4)))

production run,17,2018-03-29 14:40:25.578394,,Unit,3
production run,17,2018-03-29 14:40:25.579375,,Unit,4


Note that `key_part_3` is a function (`datetime.now`) so each row gets a new value

We can override the defaults with a fixed value:

In [30]:
BaseElements.key_part3 = datetime(2017, 1, 31)
print(row(ShortTable(143)))

production run,17,2017-01-31 00:00:00,,Unit,143


The `_clear()` function resets everything *in the target class* back to defaults.

In [31]:
BaseElements._clear()
print(row(ShortTable(12)))

,,2018-03-29 14:40:25.608772,,Unit,12


DynProps values can be fixed values, functions (`f() -> object`) or methods (`f(self) -> object`).  The first 
(and only) argument must be "self" in the second case.

The `Parent` tag identifies where the parent properties appear.  The default is before the children, but they can
appear after or even in the middle:

In [32]:
class MedTable(ShortTable):
    val_part3: Local[str] = "@"
    _: Parent
    val_part4: Local[int] = lambda self: self.val_part2 * 2

BaseElements.key_part1 = "Production Run"
BaseElements.key_part2 = 1001743
BaseElements.key_part3 = datetime.now
        
mt = MedTable(42)
print(heading(mt))
print(row(mt))
print(row(mt))

val_part3,key_part1,key_part2,key_part3,key_part4,val_part1,val_part2,val_part4
@,Production Run,1001743,2018-03-29 14:40:25.645471,,Unit,42,84
@,Production Run,1001743,2018-03-29 14:40:25.646855,,Unit,42,84


### Dynamic Properties as dictionaries
The `as_dict` function returns an ordered dictionary representation of a `DynProps` instance

In [33]:
for k, v in as_dict(mt).items():
    print(f"{k}: {v}")

val_part3: @
key_part1: Production Run
key_part2: 1001743
key_part3: 2018-03-29 14:40:25.662964
key_part4: None
val_part1: Unit
val_part2: 42
val_part4: 84


### Additional features
* **`DynProps._sql_string_delimiter`** - the delimiter to be used in `row` representation. Default: `'"'`
* **`DynProps._sql_string_delimiter_escape`** - escape character for an embedded string.  Default: `r'\"'`.  You may need
to set this to `'""'` for Oracle implementations
* **`DynProps._sql_null_text`** - representation for null (Python: `None`) in `row`.  Default is an empty string (`''`)
* **`DynProps._separator`** - `heading` and `row` separator.  Default is a tab (`'\t'`)

### `reify` function
It may be necessary to produce a representation of an object for the `row()` and `as_dict()` functions that isn't necessarily a string, meaning `str()` won't work.  In addition, `repr()` 
would convert things that don't need transformations.  IF an object has a `reify()` method,
this will be called when generating a `row` or `as_dict` output.

In [34]:
 class SpecialProp1:
    def __init__(self, parts) -> None:
        self.parts = parts

    def reify(self):
        return '-'.join(str(s) for s in self.parts) if self.parts else None

class SpecialProp2:
    def __init__(self, *parts) -> None:
        self.parts = parts

    def reify(self):
        return sum(int(p) for p in self.parts) if self.parts else None

class R1(DynProps):
    sp1: Local[SpecialProp1]
    sp2: Local[SpecialProp1]
    sp3: Local[SpecialProp2]
    sp4: Local[SpecialProp2]

r = R1()
r.sp1 = SpecialProp1(['a', 17, None])
r.sp2 = SpecialProp1([])
r.sp3 = SpecialProp2(17, -3, 100101)
r.sp4 = SpecialProp2()
print(row(r))
for k, v in as_dict(r).items():
    print(f"{k}: {v}")

a-17-None,,100115,
sp1: a-17-None
sp2: None
sp3: 100115
sp4: None


### Bypassing the reify function
There are occassions when you need to look inside a reifiable property. This can be accomplished
by suffixing a '_' to the property name

In [35]:
r.sp1_.parts.append("Alpha")
print(r.sp1)
print(r.sp1_)

a-17-None-Alpha


## Undone tasks

1) The typing within the `Global` and `Local` elements is not directly exposed.  We need to re-write the `__annotation__` elements in the constructor to allow IDE type checking.

2) SQL value escaping is minimal -- the `row` functions, in particular, need to take full advantage of SQL library escaping

3) There are no SQL injection protections in this module. May not be needed, but should be at least discussed